<a href="https://colab.research.google.com/github/liron7722/AI-Generated-Text-Detector/blob/Production/tdIdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Load CSV
df = pd.read_parquet('https://uc5cdb122338ef7262b1375676d3.dl.dropboxusercontent.com/cd/0/get/Cf4cRQyHycNLBum2ODU7nYWbAkTDr_tJeFErHL-bSNk27wrDe3N1NNFNMlUIfjTmRW_AMhfG9uOEcpcMz9ddb7yYNQRZhs6LvdUNeLTumc0PiDE9Eg98AS52yK1Q2mdq-2S0nggsgwLl0oxICQifYzz9FaJbco2iQ88uuN6YhR2UYw/file?_download_id=83887963961934459430746542227137228960885289283044250126663787175&_log_download_success=1#')


# Calculate TF-IDF
vectorizer = TfidfVectorizer(max_features=250, ngram_range=(2,4))
tfidf_matrix = vectorizer.fit_transform(df['text'])

# Convert to DataFrame
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=vectorizer.get_feature_names_out()
)

# Save to CSV
tfidf_df.to_csv('tfidf.csv', index=False)